In [1]:
import sys
sys.path.insert(0, '..')
from ModelInterface import ModelBuilder, BuildingMetadataLoader, add_points, Survey, SurveyReader
from buildingmotif.namespaces import BRICK, RDF

# Testing the Model Builder

In [2]:
builder = ModelBuilder(
    site_id="test-site",
    timezone="America/New_York",
    latitude=40.7128,
    longitude=-74.0060,
    noaa_station="KJFK"
)

/opt/anaconda3/envs/hpflex_mpc/lib/python3.11/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


In [3]:
builder.add_zone("zone1")
builder.add_window("window1", "zone1", area_value=10.5, azimuth_value=180, tilt_value=30)
builder.add_thermostat("tstat1", "zone1", stage_count=2, setpoint_deadband=1, tolerance=2, active=True, resolution = "float",unit="IP")
builder.add_hvac("hvac1", "zone1", cooling_capacity=5.0, heating_capacity=4.0, cooling_cop=3.5, heating_cop=3.0)
builder.add_space("space1", "zone1", area_value=50.0)

builder.add_zone("zone2")
builder.add_window("window2", "zone2", area_value=10.5, azimuth_value=180, tilt_value=30)
builder.add_thermostat("tstat2", "zone2", stage_count=2, setpoint_deadband=1, tolerance=2, active=True, resolution =  "float",unit="IP")
builder.add_hvac("hvac2", "zone2", cooling_capacity=5.0, heating_capacity=4.0, cooling_cop=3.5, heating_cop=3.0)
builder.add_space("space2", "zone2", area_value=50.0)

zone1
zone2


In [4]:
# Using AQL for proof of concept, this part is incomplete
add_points(builder, "../development_files/metadata-survey-variable-map.csv")

In [5]:
builder.save_model("test-brick-model.ttl")

# Testing get Metadata

In [6]:
site_metadata = BuildingMetadataLoader("test-brick-model.ttl")
site_info = site_metadata.get_site_info()
thermostat_data = site_metadata.get_thermostat_data()

In [7]:
thermostat_data

{'heat_availability': [False, False],
 'cool_availability': [True, True],
 'heat_tolerance': [-1.111111, -1.111111],
 'cool_tolerance': [1.111111, 1.111111],
 'setpoint_deadband': [0.5555556, 0.5555556],
 'active': [True, True],
 'control_group': ['urn:hpflex/test-site#tstat1_control_group',
  'urn:hpflex/test-site#tstat2_control_group'],
 'control_type_list': ['stage', 'stage'],
 'floor_area_list': [50.0, 50.0],
 'window_area_list': [10.5, 10.5],
 'azimuth_list': [180, 180],
 'tilt_list': [30, 30],
 'zone_ids': ['zone1', 'zone2'],
 'hvacs': ['hvac1', 'hvac2'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['gas', 'gas'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [5.0, 5.0],
 'heating_capacity': [4.0, 4.0],
 'cooling_cop': [3.5, 3.5],
 'heating_cop': [3.0, 3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': ['float', 'float'],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F'),
  rdflib.term.U

In [8]:
site_metadata.get_thermostat_data(for_zone='zone1')

{'heat_availability': [False],
 'cool_availability': [True],
 'heat_tolerance': [-1.111111],
 'cool_tolerance': [1.111111],
 'setpoint_deadband': [0.5555556],
 'active': [True],
 'control_group': ['urn:hpflex/test-site#tstat1_control_group'],
 'control_type_list': ['stage'],
 'floor_area_list': [50.0],
 'window_area_list': [10.5],
 'azimuth_list': [180],
 'tilt_list': [30],
 'zone_ids': ['zone1'],
 'hvacs': ['hvac1'],
 'setpoint_type': ['double'],
 'fuel_heat_list': ['gas'],
 'fuel_cool_list': ['electricity'],
 'cooling_capacity': [5.0],
 'heating_capacity': [4.0],
 'cooling_cop': [3.5],
 'heating_cop': [3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': ['float'],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F')]}

In [9]:
site_metadata.get_complete_output()

{'tz': 'America/New_York',
 'latitude': 40.7128,
 'longitude': -74.006,
 'NOAAstation': None,
 'project_id': rdflib.term.URIRef('urn:hpflex/test-site#test-site'),
 'site_id': rdflib.term.URIRef('urn:hpflex/test-site#test-site'),
 'heat_availability': [False, False],
 'cool_availability': [True, True],
 'heat_tolerance': [-1.111111, -1.111111],
 'cool_tolerance': [1.111111, 1.111111],
 'setpoint_deadband': [0.5555556, 0.5555556],
 'active': [True, True],
 'control_group': ['urn:hpflex/test-site#tstat1_control_group',
  'urn:hpflex/test-site#tstat2_control_group'],
 'control_type_list': ['stage', 'stage'],
 'floor_area_list': [50.0, 50.0],
 'window_area_list': [10.5, 10.5],
 'azimuth_list': [180, 180],
 'tilt_list': [30, 30],
 'zone_ids': ['zone1', 'zone2'],
 'hvacs': ['hvac1', 'hvac2'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['gas', 'gas'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [5.0, 5.0],
 'heating_capacity': [4.0, 4.0],
 'cooling_c

# Generating Metadata Survey

In [10]:
#Old version
# interface = Survey(
#     site_name="empty-test-site",
#     num_zones=3,
#     spaces_per_zone=2,
#     num_hvac_units=2,
#     one_hvac_per_zone=True,
#     hvac_type="hp-rtu"
# )
#interface.generate_template("./metadata-survey")

In [11]:
survey_gen = Survey(site_name = 'easy-config-test',
                             hvac_type = 'hp-rtu')
# zone_space_window list for configuration: 3 zones, with 2 spaces and 2 windows, 1 space and 2 windows, 1 space 3 windows, one hvac unit per zone
survey_gen.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)], output_path='./metadata-survey')
# survey_gen.easy_config(zone_space_window_list=[(1,1)], output_path='./metadata-survey')


In [12]:
# Custom config 
# TODO: add underscores to begining of all methods but generate template and easy config
survey_gen = Survey(site_name= 'custom-config-test',
                             hvac_type = 'vrf')
survey_gen.generate_template(
    hvacs_feed_hvacs = {'HP1': ['FCU1','FCU2','FCU3'],
                        'HP2': ['FCU1', 'FCU2', 'FCU3']},
    hvacs_feed_zones= {'FCU1': ['Zone1'],
                       'FCU2': ['Zone2'],
                       'FCU3': ['Zone3']},
    zones_contain_spaces= {'Zone1': ['Z1S1', 'Z1S2'],
                           'Zone2': ['Z1S2','Z2S3'],
                           'Zone3': ['Z3S1']},
    zones_contain_windows= {'Zone1': ['W1'],
                           'Zone2': ['W2'],
                           'Zone3': ['W3','W4','W5','W6']},
                           output_path='./metadata-survey'
)

# Reading Metadata Survey

In [13]:
generator = SurveyReader("./metadata-survey/filled-out-easy-config-test")
generator.create_model("survey-brick-model.ttl")

/opt/anaconda3/envs/bmotif/lib/python3.11/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an identity for (<class 'buildingmotif.database.tables.DepsAssociation'>, (1,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  self.bm.session.flush()
/opt/anaconda3/envs/bmotif/lib/python3.11/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an identity for (<class 'buildingmotif.database.tables.DepsAssociation'>, (2,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  self.bm.session.flush()
/opt/anaconda3/envs/bmotif/lib/python3.11/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an identity for (<class 'buildingmotif.database.tables.DepsAssociation'>, (3,), None), replacing it with new

In [14]:
generator.config['hvacs_feed_zones']

{'hvac_1': ['zone_1']}